# Aprendizado Federado para reconhecimento de contexto em dispositivos móveis
## Experimento 1 - Pré-treinamento usual com uma base de usuários e Aprendizado Federado embarcado partindo do modelo pretreinado 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import es_utils as utils

2022-03-24 15:02:09.432400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 15:02:09.466422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-24 15:02:09.466439: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-24 15:02:09.466869: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Inicialmente precisamos separar quais 'labels' usaremos. Aqui, optamos pelas 51 labels que segundo artigo do ExtraSensosy utiliza, de modo a ter bons parâmetros de comparação

In [2]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

"labels = ['label:LYING_DOWN',\n 'label:SITTING',\n 'label:FIX_walking',\n 'label:FIX_running',\n 'label:BICYCLING',\n 'label:SLEEPING',\n 'label:LAB_WORK',\n 'label:IN_CLASS',\n 'label:IN_A_MEETING',\n 'label:LOC_main_workplace',\n 'label:OR_indoors',\n 'label:OR_outside',\n 'label:IN_A_CAR',\n 'label:ON_A_BUS',\n 'label:DRIVE_-_I_M_THE_DRIVER',\n 'label:DRIVE_-_I_M_A_PASSENGER',\n 'label:LOC_home',\n 'label:FIX_restaurant',\n 'label:PHONE_IN_POCKET',\n 'label:OR_exercise',\n 'label:COOKING',\n 'label:SHOPPING',\n 'label:STROLLING',\n 'label:DRINKING__ALCOHOL_',\n 'label:BATHING_-_SHOWER',\n 'label:CLEANING',\n 'label:DOING_LAUNDRY',\n 'label:WASHING_DISHES',\n 'label:WATCHING_TV',\n 'label:SURFING_THE_INTERNET',\n 'label:AT_A_PARTY',\n 'label:AT_A_BAR',\n 'label:LOC_beach',\n 'label:SINGING',\n 'label:TALKING',\n 'label:COMPUTER_WORK',\n 'label:EATING',\n 'label:TOILET',\n 'label:GROOMING',\n 'label:DRESSING',\n 'label:AT_THE_GYM',\n 'label:STAIRS_-_GOING_UP',\n 'label:STAIRS_-_GOING

In [3]:
labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]

len(labels)

6

Separamos os 60 usuários em 5 pastas onde cada pasta considera 40 usuários selecionados aleatoriamente para treino e os 20 restantes para teste. Os dados dos 20 usuários são ainda separados em 4 arquivos *.csv* (x_train, x_test, y_train, y_test) e colocados em pastas nomeadas com o uuid do usuário.

In [4]:
#k_folds = utils.create_k_folds_n_users(5, 3, '/home/wander/OtherProjects/har_flower/sample_data')
k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

Em seguida, para cada pasta, treinamos um modelo de MLP fazendo uma otimização de hiperparâmetros e convertemos em um modelo *tflite*. Os hiperparâmetros otimizados são:

- número de camadas escondidas (*hidden layers*): 1 ou 2
- número de neurônios em cada camada escondida: 4 a 64, com um passo de tamanho 4
- constante de penalização da regularização L2 em cada camada: 1e-2, 1e-3 ou 1e-4
- taxa de aprendizado (*learning rate*) do otimizador (*Stochastic Gradient Descent*, **SGD**): 1e-1, 1e-2, 1e-3 ou 1e-4
- momentum do otimizador (**SGD**): 0.8, 0.5, 0.3 ou 0.1

In [5]:

for fold in k_folds:
    config = {
        'df_path': k_folds[fold]['40'],
        'labels': labels
    }
    
    har = utils.HAR(config)
    #har.run()
    har.hypertunning()
    
    har.mlp.model.save(f'model/saved_model_{fold}')
    converter = tf.lite.TFLiteConverter.from_saved_model(f'model/saved_model_{fold}') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'model/model_base_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

Trial 30 Complete [00h 00m 58s]
val_avg_multilabel_BA_2: 0.5335937738418579

Best val_avg_multilabel_BA_2 So Far: 0.5856737494468689
Total elapsed time: 00h 12m 24s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
3949/3949 [==============================] - 4s 911us/step - loss: 1177874.3750 - avg_multilabel_BA_2: 0.5312 - val_loss: 18357.0059 - val_avg_multilabel_BA_2: 0.5314
Epoch 2/100
3949/3949 [==============================] - 3s 819us/step - loss: 15196.8604 - avg_multilabel_BA_2: 0.5317 - val_loss: 23062.7129 - val_avg_multilabel_BA_2: 0.5318
Epoch 3/100
3949/3949 [==============================] - 3s 815us/step - loss: 15257.0137 - avg_multilabel_BA_2: 0.5319 - val_loss: 11776.5293 - val_avg_multilabel_BA_2: 0.5321
Epoch 4/100
3949/3949 [==============================] - 3s 805us/step - loss: 14225.4678 - avg_multilabel_BA_2: 0.5323 - val_loss: 6611.7837 - val_avg_multilabel_BA_2: 0.5326
Epoch 5/100
3949/3949 [==============================] - 3s 802us/step - loss: 14449.830

3949/3949 [==============================] - 3s 856us/step - loss: 10833.2158 - avg_multilabel_BA_2: 0.5451 - val_loss: 19211.6055 - val_avg_multilabel_BA_2: 0.5452
Epoch 47/100
3949/3949 [==============================] - 4s 890us/step - loss: 10562.0352 - avg_multilabel_BA_2: 0.5453 - val_loss: 9585.2227 - val_avg_multilabel_BA_2: 0.5454
Epoch 48/100
3949/3949 [==============================] - 3s 859us/step - loss: 10834.3281 - avg_multilabel_BA_2: 0.5456 - val_loss: 15879.3291 - val_avg_multilabel_BA_2: 0.5457
Epoch 49/100
3949/3949 [==============================] - 3s 857us/step - loss: 10905.7402 - avg_multilabel_BA_2: 0.5459 - val_loss: 15201.7637 - val_avg_multilabel_BA_2: 0.5460
Epoch 50/100
3949/3949 [==============================] - 3s 867us/step - loss: 10803.6895 - avg_multilabel_BA_2: 0.5461 - val_loss: 6296.4956 - val_avg_multilabel_BA_2: 0.5462
Epoch 51/100
3949/3949 [==============================] - 3s 868us/step - loss: 10638.6445 - avg_multilabel_BA_2: 0.5464 - va

3949/3949 [==============================] - 3s 817us/step - loss: 9516.2363 - avg_multilabel_BA_2: 0.5556 - val_loss: 5470.2236 - val_avg_multilabel_BA_2: 0.5558
Epoch 93/100
3949/3949 [==============================] - 3s 814us/step - loss: 9459.0674 - avg_multilabel_BA_2: 0.5560 - val_loss: 15865.0537 - val_avg_multilabel_BA_2: 0.5560
Epoch 94/100
3949/3949 [==============================] - 3s 813us/step - loss: 9337.7471 - avg_multilabel_BA_2: 0.5561 - val_loss: 9501.9893 - val_avg_multilabel_BA_2: 0.5563
Epoch 95/100
3949/3949 [==============================] - 3s 818us/step - loss: 9374.3760 - avg_multilabel_BA_2: 0.5565 - val_loss: 11567.3037 - val_avg_multilabel_BA_2: 0.5566
Epoch 96/100
3949/3949 [==============================] - 3s 819us/step - loss: 9159.8428 - avg_multilabel_BA_2: 0.5567 - val_loss: 8962.3682 - val_avg_multilabel_BA_2: 0.5568
Epoch 97/100
3949/3949 [==============================] - 3s 814us/step - loss: 9243.0820 - avg_multilabel_BA_2: 0.5570 - val_loss:

3949/3949 [==============================] - 3s 773us/step - loss: 16740.4688 - avg_multilabel_BA_2: 0.5611 - val_loss: 31684.2363 - val_avg_multilabel_BA_2: 0.5612
Epoch 39/100
3949/3949 [==============================] - 3s 775us/step - loss: 17282.5449 - avg_multilabel_BA_2: 0.5613 - val_loss: 29803.2070 - val_avg_multilabel_BA_2: 0.5613
Epoch 40/100
3949/3949 [==============================] - 3s 773us/step - loss: 16236.0684 - avg_multilabel_BA_2: 0.5614 - val_loss: 17729.5059 - val_avg_multilabel_BA_2: 0.5615
Epoch 41/100
3949/3949 [==============================] - 3s 775us/step - loss: 16526.2832 - avg_multilabel_BA_2: 0.5616 - val_loss: 6555.6562 - val_avg_multilabel_BA_2: 0.5616
Epoch 42/100
3949/3949 [==============================] - 3s 805us/step - loss: 16045.7842 - avg_multilabel_BA_2: 0.5617 - val_loss: 31438.6406 - val_avg_multilabel_BA_2: 0.5617
Epoch 43/100
3949/3949 [==============================] - 3s 806us/step - loss: 16328.8057 - avg_multilabel_BA_2: 0.5617 - v

3949/3949 [==============================] - 3s 789us/step - loss: 14267.4355 - avg_multilabel_BA_2: 0.5662 - val_loss: 7123.4121 - val_avg_multilabel_BA_2: 0.5662
Epoch 85/100
3949/3949 [==============================] - 3s 788us/step - loss: 13712.1729 - avg_multilabel_BA_2: 0.5662 - val_loss: 15081.8750 - val_avg_multilabel_BA_2: 0.5663
Epoch 86/100
3949/3949 [==============================] - 3s 797us/step - loss: 14052.4043 - avg_multilabel_BA_2: 0.5664 - val_loss: 20123.0000 - val_avg_multilabel_BA_2: 0.5664
Epoch 87/100
3949/3949 [==============================] - 3s 789us/step - loss: 14291.4658 - avg_multilabel_BA_2: 0.5664 - val_loss: 16186.3018 - val_avg_multilabel_BA_2: 0.5664
Epoch 88/100
3949/3949 [==============================] - 3s 787us/step - loss: 13265.4287 - avg_multilabel_BA_2: 0.5664 - val_loss: 12312.4883 - val_avg_multilabel_BA_2: 0.5665
Epoch 89/100
3949/3949 [==============================] - 3s 788us/step - loss: 13466.4609 - avg_multilabel_BA_2: 0.5665 - v

...

3782/3782 [==============================] - 3s 849us/step - loss: 10974.8975 - avg_multilabel_BA_2: 0.5820 - val_loss: 10574.4961 - val_avg_multilabel_BA_2: 0.5820
Epoch 48/100
3782/3782 [==============================] - 3s 846us/step - loss: 10842.6709 - avg_multilabel_BA_2: 0.5820 - val_loss: 17592.6641 - val_avg_multilabel_BA_2: 0.5820
Epoch 49/100
3782/3782 [==============================] - 3s 845us/step - loss: 10811.5381 - avg_multilabel_BA_2: 0.5820 - val_loss: 16932.9355 - val_avg_multilabel_BA_2: 0.5820
Epoch 50/100
3782/3782 [==============================] - 3s 835us/step - loss: 10778.3174 - avg_multilabel_BA_2: 0.5820 - val_loss: 4738.9189 - val_avg_multilabel_BA_2: 0.5820
Epoch 51/100
3782/3782 [==============================] - 3s 835us/step - loss: 10319.7168 - avg_multilabel_BA_2: 0.5820 - val_loss: 15954.0732 - val_avg_multilabel_BA_2: 0.5820
Epoch 52/100
3782/3782 [==============================] - 3s 850us/step - loss: 10302.4268 - avg_multilabel_BA_2: 0.5820 - v

3782/3782 [==============================] - 3s 830us/step - loss: 9107.8301 - avg_multilabel_BA_2: 0.5829 - val_loss: 13705.0957 - val_avg_multilabel_BA_2: 0.5829
Epoch 94/100
3782/3782 [==============================] - 3s 834us/step - loss: 8529.6709 - avg_multilabel_BA_2: 0.5829 - val_loss: 15698.6895 - val_avg_multilabel_BA_2: 0.5829
Epoch 95/100
3782/3782 [==============================] - 3s 833us/step - loss: 8777.4092 - avg_multilabel_BA_2: 0.5830 - val_loss: 10284.5918 - val_avg_multilabel_BA_2: 0.5830
Epoch 96/100
3782/3782 [==============================] - 3s 834us/step - loss: 8734.0391 - avg_multilabel_BA_2: 0.5830 - val_loss: 11611.0469 - val_avg_multilabel_BA_2: 0.5830
Epoch 97/100
3782/3782 [==============================] - 3s 832us/step - loss: 8642.5508 - avg_multilabel_BA_2: 0.5830 - val_loss: 13035.1533 - val_avg_multilabel_BA_2: 0.5830
Epoch 98/100
3782/3782 [==============================] - 3s 828us/step - loss: 8420.4346 - avg_multilabel_BA_2: 0.5830 - val_lo

3782/3782 [==============================] - 3s 853us/step - loss: 12500.8936 - avg_multilabel_BA_2: 0.5832 - val_loss: 19191.0723 - val_avg_multilabel_BA_2: 0.5832
Epoch 40/100
3782/3782 [==============================] - 3s 863us/step - loss: 13006.5957 - avg_multilabel_BA_2: 0.5832 - val_loss: 14229.6426 - val_avg_multilabel_BA_2: 0.5832
Epoch 41/100
3782/3782 [==============================] - 3s 858us/step - loss: 12737.0303 - avg_multilabel_BA_2: 0.5832 - val_loss: 20926.1641 - val_avg_multilabel_BA_2: 0.5832
Epoch 42/100
3782/3782 [==============================] - 3s 860us/step - loss: 12714.8916 - avg_multilabel_BA_2: 0.5833 - val_loss: 8615.1582 - val_avg_multilabel_BA_2: 0.5833
Epoch 43/100
3782/3782 [==============================] - 3s 854us/step - loss: 12745.0986 - avg_multilabel_BA_2: 0.5833 - val_loss: 20988.9492 - val_avg_multilabel_BA_2: 0.5833
Epoch 44/100
3782/3782 [==============================] - 3s 858us/step - loss: 12767.2871 - avg_multilabel_BA_2: 0.5833 - v

3782/3782 [==============================] - 3s 853us/step - loss: 11111.8369 - avg_multilabel_BA_2: 0.5840 - val_loss: 18245.8242 - val_avg_multilabel_BA_2: 0.5840
Epoch 86/100
3782/3782 [==============================] - 3s 856us/step - loss: 11104.7422 - avg_multilabel_BA_2: 0.5840 - val_loss: 19706.2617 - val_avg_multilabel_BA_2: 0.5840
Epoch 87/100
3782/3782 [==============================] - 3s 877us/step - loss: 11147.9805 - avg_multilabel_BA_2: 0.5840 - val_loss: 8549.3770 - val_avg_multilabel_BA_2: 0.5840
Epoch 88/100
3782/3782 [==============================] - 3s 844us/step - loss: 11036.7705 - avg_multilabel_BA_2: 0.5840 - val_loss: 13199.3213 - val_avg_multilabel_BA_2: 0.5840
Epoch 89/100
3782/3782 [==============================] - 3s 881us/step - loss: 11425.1660 - avg_multilabel_BA_2: 0.5840 - val_loss: 13345.9570 - val_avg_multilabel_BA_2: 0.5840
Epoch 90/100
3782/3782 [==============================] - 3s 834us/step - loss: 11320.9785 - avg_multilabel_BA_2: 0.5841 - v

INFO:tensorflow:Assets written to: model/saved_model_fold_4/assets
2022-03-24 16:22:17.239764: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-24 16:22:17.239782: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-24 16:22:17.239786: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-24 16:22:17.239956: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model_fold_4
2022-03-24 16:22:17.240904: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-24 16:22:17.240916: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model_fold_4
2022-03-24 16:22:17.243046: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-24 16:22:17.270321: I tensorflow/cc/saved_model/loader.

Com os modelos salvos, o restante do experimento segue em ambiente Android